In [1]:
import pandas as pd
import numpy as np
#url = 'https://raw.githubusercontent.com/mattharrison/datasets/master/data/2020-jetbrains-python-survey.csv'
url = 'https://github.com/mattharrison/datasets/raw/master/data/2020-jetbrains-python-survey.csv'
df = pd.read_csv(url)

/home/anggi/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (152) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
df.head()

,is.python.main,other.lang.None,other.lang.Java,other.lang.JavaScript,other.lang.C/C++,other.lang.PHP,other.lang.C#,other.lang.Ruby,other.lang.Bash / Shell,other.lang.Objective-C,...,job.role.Technical support,job.role.Data analyst,job.role.Business analyst,job.role.Team lead,job.role.Product manager,job.role.CIO / CEO / CTO,job.role.Systems analyst,job.role.Other,age,country.live
0,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bash / Shell,NaN,...,NaN,NaN,Business analyst,NaN,NaN,NaN,NaN,NaN,30–39,NaN
1,Yes,NaN,Java,JavaScript,NaN,NaN,C#,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21–29,India
2,Yes,NaN,NaN,NaN,C/C++,NaN,NaN,NaN,Bash / Shell,NaN,...,Technical support,Data analyst,NaN,Team lead,NaN,NaN,NaN,NaN,30–39,United States
3,Yes,NaN,NaN,JavaScript,NaN,NaN,NaN,NaN,Bash / Shell,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Yes,NaN,Java,JavaScript,C/C++,NaN,NaN,NaN,Bash / Shell,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21–29,Italy


In [3]:
import collections
counter = collections.defaultdict(list)
for col in sorted(df.columns):
    period_count = col.count('.')
    if period_count >= 2:
        part_end = 2
    else:
        part_end = 1
    parts = col.split('.')[:part_end]
    counter['.'.join(parts)].append(col)
uniq_cols = []
for cols in counter.values():
    if len(cols) == 1:
        uniq_cols.extend(cols)

In [4]:
df = (df
 [uniq_cols]
 .rename(columns=lambda c: c.replace('.', '_'))
 .assign(age=lambda df_:df_.age.str.slice(0,2).astype(float)
             .astype('Int64'),
         are_you_datascientist=lambda df_:df_.are_you_datascientist
             .replace({'Yes': True, 'No': False, np.nan: False}),
         company_size=lambda df_:df_.company_size.replace({
             'Just me': 1, 'Not sure': np.nan, 
             'More than 5,000': 5000, '2–10': 2, '11–50':11,
             '51–500': 51, '501–1,000':501,
             '1,001–5,000':1001}).astype('Int64'),
         country_live=lambda df_:df_.country_live.astype('category'),
         employment_status=lambda df_:df_.employment_status
              .fillna('Other').astype('category'),
         is_python_main=lambda df_:df_.is_python_main
              .astype('category'),
         team_size=lambda df_:df_.team_size
             .str.split(r'-', n=1, expand=True)
             .iloc[:,0].replace('More than 40 people', 41)
             .where(df_.company_size!=1, 1).astype(float),
         years_of_coding=lambda df_:df_.years_of_coding
             .replace('Less than 1 year', .5).str.extract(r'(\d+)')
             .astype(float),
         python_years=lambda df_:df_.python_years
             .replace('Less than 1 year', .5).str.extract(r'(\d+)')
             .astype(float),
         python3_ver=lambda df_:df_.python3_version_most
              .str.replace('_', '.').str.extract(r'(\d\.\d)')
              .astype(float),
         use_python_most=lambda df_:df_.use_python_most
              .fillna('Unknown')
        )
    .drop(columns=['python2_version_most'])
)

In [5]:
df.head()

,age,are_you_datascientist,company_size,country_live,employment_status,first_learn_about_main_ide,how_often_use_main_ide,ide_main,is_python_main,job_team,main_purposes,missing_features_main_ide,nps_main_ide,python_years,python3_version_most,several_projects,team_size,use_python_most,years_of_coding,python3_ver
0,30,False,1,NaN,Partially employed by a company / organization,Conference / User Group,Weekly,PyCharm Community Edition,Yes,Work as an external consultant or trainer,For work,"No, it has all the features I need",3.0,3.0,Python 3_7,"Yes, I work on many different projects",1.0,Unknown,1.0,3.7
1,21,True,5000,India,Fully employed by a company / organization,School / University,Daily,VS Code,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",8.0,3.0,Python 3_6,"Yes, I work on one main and several side projects",2.0,Software prototyping,3.0,3.6
2,30,False,5000,United States,Fully employed by a company / organization,Friend / Colleague,Daily,Vim,Yes,Work on your own project(s) independently,Both for work and personal,"No, it has all the features I need",10.0,3.0,Python 3_6,"Yes, I work on one main and several side projects",NaN,DevOps / System administration / Writing autom...,3.0,3.6
3,<NA>,False,<NA>,NaN,Other,Friend / Colleague,Daily,PyCharm Professional Edition,Yes,NaN,Both for work and personal,Yes – Please list:,10.0,11.0,Python 3_8,"Yes, I work on many different projects",1.0,Web development,11.0,3.8
4,21,False,<NA>,Italy,Student,Search engines,Daily,VS Code,Yes,Work on your own project(s) independently,"For personal, educational or side projects","No, it has all the features I need",10.0,1.0,Python 3_8,"Yes, I work on one main and several side projects",1.0,Web development,NaN,3.8


### Pivot table

In [6]:
df.columns

Index(['age', 'are_you_datascientist', 'company_size', 'country_live',
       'employment_status', 'first_learn_about_main_ide',
       'how_often_use_main_ide', 'ide_main', 'is_python_main', 'job_team',
       'main_purposes', 'missing_features_main_ide', 'nps_main_ide',
       'python_years', 'python3_version_most', 'several_projects', 'team_size',
       'use_python_most', 'years_of_coding', 'python3_ver'],
      dtype='object')

In [21]:
df.age = df.age.fillna(0)
pivot_df = df.pivot_table(index='country_live', columns='employment_status',
                          values='age', aggfunc=np.mean)

In [25]:
pivot_df.head()

employment_status,Freelancer (a person pursuing a profession without a long-term commitment to any one employer),Fully employed by a company / organization,Other,Partially employed by a company / organization,Retired,"Self-employed (a person earning income directly from one's own business, trade, or profession)",Student,Working student
country_live,,,,,,,,
Algeria,33.285714,28.344828,43.333333,25.181818,60.000000,28.411765,20.437500,21.000000
Argentina,34.954545,30.209524,40.000000,30.250000,55.000000,29.571429,22.913043,23.200000
Armenia,30.000000,23.842105,60.000000,24.000000,40.000000,24.000000,NaN,NaN
Australia,30.818182,33.074468,33.100000,32.000000,48.285714,39.218750,24.828571,22.470588
Austria,35.857143,31.318182,30.000000,29.000000,60.000000,34.615385,20.842105,23.812500


### Crosstab

In [26]:
# NOTE: pay attention how crosstab can handle null values better than pivot_table
crosstab_df = pd.crosstab(index=df.country_live, columns=df.employment_status, 
                          values=df.age, aggfunc=np.mean).fillna(0)

In [27]:
crosstab_df.head()

employment_status,Freelancer (a person pursuing a profession without a long-term commitment to any one employer),Fully employed by a company / organization,Other,Partially employed by a company / organization,Retired,"Self-employed (a person earning income directly from one's own business, trade, or profession)",Student,Working student
country_live,,,,,,,,
Algeria,33.285714,28.344828,43.333333,25.181818,60.000000,28.411765,20.437500,21.000000
Argentina,34.954545,30.209524,40.000000,30.250000,55.000000,29.571429,22.913043,23.200000
Armenia,30.000000,23.842105,60.000000,24.000000,40.000000,24.000000,0.000000,0.000000
Australia,30.818182,33.074468,33.100000,32.000000,48.285714,39.218750,24.828571,22.470588
Austria,35.857143,31.318182,30.000000,29.000000,60.000000,34.615385,20.842105,23.812500


### Multiple aggregasion in pivot_table and crosstab

In [31]:
pivot_multiple_df = df.pivot_table(index='country_live', columns='employment_status',
                                   values='age', aggfunc=[np.mean, np.min]).fillna(0)

In [32]:
pivot_multiple_df.head()

mean  \
employment_status Freelancer (a person pursuing a profession without a long-term commitment to any one employer)   
country_live                                                                                                       
Algeria                                                    33.285714                                               
Argentina                                                  34.954545                                               
Armenia                                                    30.000000                                               
Australia                                                  30.818182                                               
Austria                                                    35.857143                                               

                                                                         \
employment_status Fully employed by a company / organization      Other   
country_live                                                              
Algeria                                            28.344828  43.333333   
Argentina                                          30.209524  40.000000   
Armenia                                            23.842105  60.000000   
Australia                                          33.074468  33.100000   
Austria                                            31.318182  30.000000   

                                                                             \
employment_status Partially employed by a company / organization    Retired   
country_live                                                                  
Algeria                                                25.181818  60.000000   
Argentina                                              30.250000  55.000000   
Armenia                                                24.000000  40.000000   
Australia                                              32.000000  48.285714   
Austria                                                29.000000  60.000000   

                                                                                                                  \
employment_status Self-employed (a person earning income directly from one's own business, trade, or profession)   
country_live                                                                                                       
Algeria                                                    28.411765                                               
Argentina                                                  29.571429                                               
Armenia                                                    24.000000                                               
Australia                                                  39.218750                                               
Austria                                                    34.615385                                               

                                              \
employment_status    Student Working student   
country_live                                   
Algeria            20.437500       21.000000   
Argentina          22.913043       23.200000   
Armenia             0.000000        0.000000   
Australia          24.828571       22.470588   
Austria            20.842105       23.812500   

                                                                                                            amin  \
employment_status Freelancer (a person pursuing a profession without a long-term commitment to any one employer)   
country_live                                                                                                       
Algeria                                                         21.0                                               
Argentina                                                       18.0                                               
Armenia                                            

In [34]:
crosstab_multiple_df = pd.crosstab(index=df.country_live, columns=df.employment_status, 
                                   values=df.age, aggfunc=[np.mean, np.min]).fillna(0)

In [35]:
crosstab_multiple_df.head()

mean  \
employment_status Freelancer (a person pursuing a profession without a long-term commitment to any one employer)   
country_live                                                                                                       
Algeria                                                    33.285714                                               
Argentina                                                  34.954545                                               
Armenia                                                    30.000000                                               
Australia                                                  30.818182                                               
Austria                                                    35.857143                                               

                                                                         \
employment_status Fully employed by a company / organization      Other   
country_live                                                              
Algeria                                            28.344828  43.333333   
Argentina                                          30.209524  40.000000   
Armenia                                            23.842105  60.000000   
Australia                                          33.074468  33.100000   
Austria                                            31.318182  30.000000   

                                                                             \
employment_status Partially employed by a company / organization    Retired   
country_live                                                                  
Algeria                                                25.181818  60.000000   
Argentina                                              30.250000  55.000000   
Armenia                                                24.000000  40.000000   
Australia                                              32.000000  48.285714   
Austria                                                29.000000  60.000000   

                                                                                                                  \
employment_status Self-employed (a person earning income directly from one's own business, trade, or profession)   
country_live                                                                                                       
Algeria                                                    28.411765                                               
Argentina                                                  29.571429                                               
Armenia                                                    24.000000                                               
Australia                                                  39.218750                                               
Austria                                                    34.615385                                               

                                              \
employment_status    Student Working student   
country_live                                   
Algeria            20.437500       21.000000   
Argentina          22.913043       23.200000   
Armenia             0.000000        0.000000   
Australia          24.828571       22.470588   
Austria            20.842105       23.812500   

                                                                                                            amin  \
employment_status Freelancer (a person pursuing a profession without a long-term commitment to any one employer)   
country_live                                                                                                       
Algeria                                                         21.0                                               
Argentina                                                       18.0                                               
Armenia                                            

### Transforming